In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

from keras import Sequential,losses
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from keras.optimizers import SGD,Adam
from keras.utils import np_utils

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

/kaggle/input/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data.csv
/kaggle/input/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data/A_Z Handwritten Data.csv


In [4]:
df=pd.read_csv("/kaggle/input/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data.csv")

In [5]:
print("Count of Images Before deduplicating process",len(df))
df.drop_duplicates(inplace=True)
print("Count of Images After deduplicating process",len(df))

Count of Images Before deduplicating process 372450
Count of Images After deduplicating process 201095


In [6]:
datagen = ImageDataGenerator(width_shift_range=2,height_shift_range=2,validation_split=0.2)
X = df.loc[:,"0.1":]
y = df.loc[:,"0"]
X=X.values.reshape(len(X),28,28,1)
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.90)
X_train = X_train/255
X_test = X_test/255
y_train=np_utils.to_categorical(y_train)

In [7]:
model=Sequential() # A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.
model.add(Conv2D(8,(3,3),activation="relu")) #2D convolution layer
model.add(MaxPooling2D(strides=1))
model.add(Conv2D(16,(3,3),activation="relu"))
model.add(MaxPooling2D(strides=1))
model.add(Conv2D(24,(3,3),activation="relu"))
model.add(MaxPooling2D(strides=1))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(26,activation="softmax"))
model.compile(loss=losses.categorical_crossentropy,metrics=["accuracy"],optimizer=SGD())

In [8]:
checkpoint = ModelCheckpoint(filepath="model_sgd.pkl",monitor="val_accuracy",verbose=2,save_best_only=True,mode="max")
model.fit(datagen.flow(X_train, y_train, batch_size=256,subset="training"),
          validation_data=datagen.flow(X_train, y_train, batch_size=256,subset="validation"),
          epochs=50,batch_size=256,verbose=2,callbacks=[checkpoint])

Epoch 1/50

Epoch 1: val_accuracy improved from -inf to 0.84957, saving model to model_sgd.pkl
566/566 - 54s - loss: 1.0609 - accuracy: 0.7010 - val_loss: 0.5426 - val_accuracy: 0.8496 - 54s/epoch - 96ms/step
Epoch 2/50

Epoch 2: val_accuracy improved from 0.84957 to 0.91820, saving model to model_sgd.pkl
566/566 - 51s - loss: 0.4207 - accuracy: 0.8815 - val_loss: 0.2940 - val_accuracy: 0.9182 - 51s/epoch - 90ms/step
Epoch 3/50

Epoch 3: val_accuracy improved from 0.91820 to 0.93613, saving model to model_sgd.pkl
566/566 - 52s - loss: 0.2471 - accuracy: 0.9306 - val_loss: 0.2219 - val_accuracy: 0.9361 - 52s/epoch - 91ms/step
Epoch 4/50

Epoch 4: val_accuracy improved from 0.93613 to 0.95505, saving model to model_sgd.pkl
566/566 - 52s - loss: 0.1804 - accuracy: 0.9494 - val_loss: 0.1593 - val_accuracy: 0.9551 - 52s/epoch - 91ms/step
Epoch 5/50

Epoch 5: val_accuracy improved from 0.95505 to 0.96223, saving model to model_sgd.pkl
566/566 - 51s - loss: 0.1490 - accuracy: 0.9578 - val_los

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, None, None, 8)     80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, None, None, 8)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 16)    1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, None, None, 16)   0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, None, None, 24)    3480      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, None, None, 24)   0

In [10]:
y_predict=model.predict(X_test)
y_predict = np.argmax(y_predict,axis=1)
accuracy_score(y_predict,y_test)

629/629 [==============================] - 2s 2ms/step


0.9839383391347588

In [11]:
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       750
           1       0.98      0.97      0.98       504
           2       0.99      0.98      0.99      1296
           3       0.92      0.97      0.95       519
           4       0.98      0.97      0.97       606
           5       0.96      0.96      0.96       111
           6       0.96      0.95      0.96       316
           7       0.98      0.95      0.97       402
           8       0.98      0.98      0.98       100
           9       0.97      0.97      0.97       452
          10       0.96      0.96      0.96       323
          11       0.99      0.98      0.98       632
          12       0.99      0.99      0.99      1019
          13       0.99      0.98      0.98      1052
          14       1.00      0.99      0.99      2877
          15       0.99      0.99      0.99      1025
          16       0.97      0.99      0.98       335
          17       0.97    

In [12]:
import shutil
shutil.make_archive("model_sgd.pkl", 'zip', "/kaggle/working/model_sgd.pkl")

'/kaggle/working/model_sgd.pkl.zip'

In [16]:
from keras.models import load_model
model1=load_model("/kaggle/working/model_sgd.pkl")

In [17]:
y_predict=model1.predict(X_test)
y_predict = np.argmax(y_predict,axis=1)
accuracy_score(y_predict,y_test)

629/629 [==============================] - 1s 2ms/step


0.9847339632023868

In [18]:
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       751
           1       0.98      0.97      0.98       505
           2       0.99      0.99      0.99      1293
           3       0.94      0.97      0.96       530
           4       0.96      0.99      0.97       584
           5       0.96      0.97      0.97       110
           6       0.95      0.97      0.96       305
           7       0.97      0.97      0.97       389
           8       0.98      0.98      0.98       100
           9       0.96      0.99      0.98       435
          10       0.98      0.95      0.96       334
          11       0.99      0.98      0.99       626
          12       1.00      0.99      0.99      1028
          13       0.99      0.98      0.99      1053
          14       1.00      0.99      0.99      2881
          15       0.99      1.00      0.99      1022
          16       0.97      0.97      0.97       341
          17       0.98    